In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam
import joblib


ModuleNotFoundError: No module named 'tensorflow'

In [1]:
!pip install pandas numpy scikit-learn tensorflow keras


^C


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam
import joblib


In [ ]:
# Define paths
data_dir = 'F:/MLpractrice/chatbot_arena/'
train_path = os.path.join(data_dir, 'train.csv')
test_path = os.path.join(data_dir, 'test.csv')

# Load datasets
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

# Display the first few rows of the train dataset
train_df.head()

# Check for missing values
print(train_df.isnull().sum())
print(test_df.isnull().sum())

# Fill missing values if any
train_df = train_df.dropna()
test_df = test_df.dropna()

# Combine responses into one feature
train_df['text'] = train_df['response_1'] + " " + train_df['response_2']
test_df['text'] = test_df['response_1'] + " " + test_df['response_2']

# Extract features and labels
X_train = train_df['text']
y_train = train_df['preferred_response']
X_test = test_df['text']

# Convert labels to numerical values
y_train = y_train.map({'response_1': 0, 'response_2': 1})


In [ ]:
# Vectorize text data
vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)


In [ ]:
# Split the data into training and validation sets
X_train_tfidf, X_val_tfidf, y_train, y_val = train_test_split(X_train_tfidf, y_train, test_size=0.2, random_state=42)

# Build Logistic Regression model
model = LogisticRegression(max_iter=1000)
model.fit(X_train_tfidf, y_train)

# Evaluate the model
y_val_pred = model.predict(X_val_tfidf)
accuracy = accuracy_score(y_val, y_val_pred)
print(f'Validation Accuracy: {accuracy:.2f}')
print(classification_report(y_val, y_val_pred))

# Make predictions on the test dataset
test_predictions = model.predict(X_test_tfidf)


In [ ]:
# Save the model and vectorizer
joblib.dump(model, 'F:/MLpractrice/chatbot_arena/chatbot_preference_model.pkl')
joblib.dump(vectorizer, 'F:/MLpractrice/chatbot_arena/tfidf_vectorizer.pkl')

print("Model and vectorizer saved successfully!")


In [ ]:
# Prepare the submission file
submission = pd.DataFrame({'id': test_df['id'], 'preferred_response': test_predictions})
submission['preferred_response'] = submission['preferred_response'].map({0: 'response_1', 1: 'response_2'})
submission.to_csv('F:/MLpractrice/chatbot_arena/submission.csv', index=False)

print("Submission file created successfully!")
